In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import SimpleRNN, LSTM, Dense, Dropout, Masking, Embedding, Dense, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input

In [ ]:
df_train = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv')
df_details = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/asset_details.csv')
df_extest = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/example_test.csv')
df_sup = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv')
df_exsamp = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/example_sample_submission.csv')

In [ ]:
print("// TRAINING DATA //")
print()
print(df_train.head(5))
print()
print("-----------")
print("// SUPLEMENTAL TRAIN //")
print()
print(df_sup.head(5))
print()
print("-----------")
print("// ASSET DETAILS //")
print()
print(df_details.head(5))
print()
print("-----------")
print("// EXAMPLE TEST //")
print()
print(df_extest.head(5))
print()
print("-----------")
print("// EXAMPLE SAMPLE //")
print()
print(df_exsamp.head(5))
print()
print("-----------")

# EDA & PREPROCESSING

In [ ]:
df_train

In [ ]:
df_train.isnull().sum()

In [ ]:
bnc = df_train[df_train['Asset_ID']==0]
btc = df_train[df_train['Asset_ID']==1]
bcs = df_train[df_train['Asset_ID']==2]
crd = df_train[df_train['Asset_ID']==3]
dog = df_train[df_train['Asset_ID']==4]
eos = df_train[df_train['Asset_ID']==5]
eth = df_train[df_train['Asset_ID']==6]
etc = df_train[df_train['Asset_ID']==7]
iot = df_train[df_train['Asset_ID']==8]
ltc = df_train[df_train['Asset_ID']==9]
mkr = df_train[df_train['Asset_ID']==10]
mnr = df_train[df_train['Asset_ID']==11]
stl = df_train[df_train['Asset_ID']==12]
trn = df_train[df_train['Asset_ID']==13]

In [ ]:
bnc = bnc.reindex(range(bnc.index[0],bnc.index[-1]+60,60),method='pad')
btc = btc.reindex(range(btc.index[0],btc.index[-1]+60,60),method='pad')
bcs = bcs.reindex(range(bcs.index[0],bcs.index[-1]+60,60),method='pad')
crd = crd.reindex(range(crd.index[0],crd.index[-1]+60,60),method='pad')
dog = dog.reindex(range(dog.index[0],dog.index[-1]+60,60),method='pad')
eos = eos.reindex(range(eos.index[0],eos.index[-1]+60,60),method='pad')
eth = eth.reindex(range(eth.index[0],eth.index[-1]+60,60),method='pad')
etc = etc.reindex(range(etc.index[0],etc.index[-1]+60,60),method='pad')
iot = iot.reindex(range(iot.index[0],iot.index[-1]+60,60),method='pad')
ltc = ltc.reindex(range(ltc.index[0],ltc.index[-1]+60,60),method='pad')
mkr = mkr.reindex(range(mkr.index[0],mkr.index[-1]+60,60),method='pad')
mnr = mnr.reindex(range(mnr.index[0],mnr.index[-1]+60,60),method='pad')
stl = stl.reindex(range(stl.index[0],stl.index[-1]+60,60),method='pad')
trn = trn.reindex(range(trn.index[0],trn.index[-1]+60,60),method='pad')


In [ ]:
df_train

In [ ]:
df_train['Target'] = df_train.groupby(['Asset_ID'], sort=False)['Target'].apply(lambda x: x.fillna(x.mean()))

In [ ]:
df_train

In [ ]:
train = df_train[:100000]
train

In [ ]:
y_train = train.Target.copy()
y_train.shape

# CREATE THE MODEL

In [ ]:
x_train = train.drop(['Target','Asset_ID','Open','High','Low'],axis = 1)
print(x_train.shape)
print(x_train.head(5))

In [ ]:
x_train = x_train.to_numpy().reshape(-1,5,1)
x_train.shape

In [ ]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(x_train, y_train, test_size = 0.25, random_state=2)

In [ ]:
from keras.layers import Dense, LSTM, Dropout, GRU
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
# The GRU architecture
model = Sequential()
# First GRU layer with Dropout regularisation
model.add(GRU(units=50, return_sequences=True, input_shape=(train_x.shape[1],1), activation='tanh'))
model.add(Dropout(0.2))
# Second GRU layer
model.add(GRU(units=50, return_sequences=True, input_shape=(train_x.shape[1],1), activation='tanh'))
model.add(Dropout(0.2))
# Third GRU layer
model.add(GRU(units=50, return_sequences=True, input_shape=(train_x.shape[1],1), activation='tanh'))
model.add(Dropout(0.2))
# Fourth GRU layer
model.add(GRU(units=50, activation='tanh'))
model.add(Dropout(0.2))
# The output layer
model.add(Dense(units=1))

In [ ]:
from keras.metrics import accuracy
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = optimizer, loss='mse', metrics = ['mae'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                          mode="min",
                          patience=5)
model.summary()

In [ ]:
history=model.fit(x_train,y_train, batch_size=64,epochs=5,validation_data=(val_x,val_y), callbacks=[early_stopping], shuffle=True)

In [ ]:
# Diffining Figure
import matplotlib.pyplot as plt
figure = plt.figure(figsize=(20,7))

#Adding Subplot 1 (For Accuracy)
figure.add_subplot(121)

plt.plot(history.epoch,history.history['mae'],label = "mae") # Accuracy curve for training set
plt.plot(history.epoch,history.history['val_mae'],label = "val_mae") # Accuracy curve for validation set

plt.title("MAE Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("MAE",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()

#Adding Subplot 1 (For Loss)
figure.add_subplot(122)

plt.plot(history.epoch,history.history['loss'],label="loss") # Loss curve for training set
plt.plot(history.epoch,history.history['val_loss'],label="val_loss") # Loss curve for validation set

plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()

plt.show()

In [ ]:
mantap = model.predict(val_x)
val_y, mantap

In [ ]:
import gresearch_crypto
env = gresearch_crypto.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:
index = 0
for (test_df, sample_prediction_df) in iter_test:
#     x_train = train.drop(['Target','Asset_ID','Open','High','Low'],axis = 1)
    test_df = test_df.drop(['Asset_ID','Open','High','Low','row_id'],axis = 1)
    test_df = test_df.to_numpy().reshape(-1,5,1)
    test_y = model.predict(test_df)
    sample_prediction_df['Target'] = 0
    sample_prediction_df['Target'] = test_y
#     print(test_df.shape)
#     print(sample_prediction_df.shape)
#     print(sample_prediction_df)
    env.predict(sample_prediction_df)

    index += 1